In [27]:
import pandas as pd

df = pd.read_csv("gov.csv")
df = df.fillna("")

# Combine text features for ML
df["combined_text"] = (
    df["scheme_name"] + " " +
    df["benefits"] + " " +
    df["eligibility"] + " " +
    df["tags"]
)

In [32]:
def filter_schemes(profile, df):
    filtered = df.copy()
    
    if profile.get("category"):
        filtered = filtered[filtered["schemeCategory"].str.contains(profile["category"], case=False, na=False)]
    if profile.get("location"):
        filtered = filtered[filtered["level"].str.contains(profile["location"], case=False, na=False)]
    if profile.get("income"):
        # Example rule: filter income-based schemes
        filtered = filtered[filtered["eligibility"].str.contains("income", case=False, na=False)]
    
    return filtered

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
X = vectorizer.fit_transform(df["combined_text"])

def recommend(profile_text, top_n=5):
    user_vec = vectorizer.transform([profile_text])
    sim = cosine_similarity(user_vec, X).flatten()
    idx = sim.argsort()[-top_n:][::-1]
    return df.iloc[idx][["scheme_name", "benefits", "eligibility"]]

In [30]:
conda install -c conda-forge fastapi uvicorn

Jupyter detected...
2 channel Terms of Service accepted
Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
doneecting package metadata (repodata.json): - 
doneing environment: - 

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [33]:
# app.py
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd

# Load your schemes data
df = pd.read_csv("gov.csv")

# Define user profile schema
class Profile(BaseModel):
    age: int | None = None
    location: str | None = None
    income: float | None = None
    category: str | None = None
    occupation: str | None = None

app = FastAPI()

@app.get("/")
def home():
    return {"message": "Welcome to SchemeSarthi API 🚀"}

@app.post("/recommend")
def recommend(profile: Profile):
    # Basic filtering logic
    filtered = df.copy()
    if profile.category:
        filtered = filtered[filtered["schemeCategory"].str.contains(profile.category, case=False, na=False)]
    if profile.location:
        filtered = filtered[filtered["level"].str.contains(profile.location, case=False, na=False)]
    
    return filtered.head(5).to_dict(orient="records")